In [2]:
%run training_functions.ipynb

['train', 'validation', 'test']
['train', 'validation', 'test']
In, Out, get_ipython, exit, quit, open, accelerate, datasets, evaluate, math, np, peft, pickle, pytest, ipytest, pd, transformers, torch, load_dataset, load_dataset_builder, get_dataset_split_names, get_dataset_config_names, LoftQConfig, LoraConfig, get_peft_model, AutoModelForCausalLM, AutoTokenizer, pipeline, TrainingArguments, Trainer, SFTTrainer, SFTConfig, @py_builtins, @pytest_ar, model_from_pkl, ds_gst1_train, ds_gst1_test, ds_gst2_train, ds_gst2_test, search_with_strings, map_data, print_trainable_parameters, metric, comp_metrics_output, compute_metrics, make_trainer, get_dataframe, get_training_output, 

## Dataset value example 
{'record': 'NZ_LIXJ01000001.1', 'text': 'CGACTCCGCACAGGCCCTAGGCGTGGTCCACGCATCGAGCGCGTGCGCCGTGGCATACACCCGGGCCACCCGTACGACCCTTCCGGCCGCCCCGTGTGCGCCCTCGTCGCCGAGTCCGCTCCGGCGCGCGCGGGCGCACTGCTGTTCCGCGCGGTGTCATGCGCTCCTCCCGTCCCGACATGCGCCTTTTTGCAGGGGAGAGGGAGGGGTAGGGTCGCGCGTACGCCTAGCACGGGGGAACGGTCTTGGGGGACGCAGTGGACGTACGGGTGGCGGCGATCGCCAGTCTCACACCGCTCGAAGAACTCGACAGCGATCCGTTCCTCGTGGACACCCGCAGCCAGCACGACATGTGCGCACGCTGGGCCGCGGACAAGGGGTACGTCGTCACCCGCCAGCTCCGCTTCTACGGGCTGCGCCCCGATCACCACGCCCTGTGGAGCGATGTCGAGAGCGGCGACGTCGAGGTGTTCGTCGCCGCCAACGACCGGGTGCTGGAACGGGCCCTCACGTCGGTCGCGGAGTTCGCCGCGGAGTGCGCGCGGCGCGGCGTGCGTCTGGAGATCGCCGGGCTGGACGAGCCGCCGTACAACGCCGGCACGAAGGCGAGCGTGCATCGCAGGCTCTCCATGCCCACCGCCGGTTACGACGGCCGCTGACGCACTCCGCCTCCACTGCCGGCTCGCGGCCCCTGCCCTCGTCCCGGCCCCGTCCCTTCCGCGGCTTGCCGTCGGGCTGTGACAGGCTGGGGGACAGGGTCCGCAGCGGCGGGGCCCGGACGTGAGGTGGTTGAGGCGTGGGTGACGGGCGATGGCGAACGGTCGGCAACGCCCTGATGCGAGTGGCCGTGGTGTGGGCGGTCTCCACGCTCACGATGCTGGTGCTCGCCGGGGTGCTGCCGGAGTTCCGGCTCCAGTCGGACGACGGCGACAGCATCACCAAGACCGCGTTCACCGCGGCCTGGGGCGCCGGAGCGTTCGGCCTGCTCTCCGCACTCGTC'}

In [3]:
# let's try prompting the fine-tuned models
def continued_convo(model, tokenizer, message, convo=None):
    #this will be used to provide models with context from previous conversations
    #this will also allow multiple messages to be consecutively sent to the same model
    pipe=pipeline("text-generation", device_map="auto", max_new_tokens=512, model=model, tokenizer=tokenizer)
    if convo==None:
        messages=[{"role": "system", "content": "You are a helpful assistant"}]
    else:
        messages=convo
    messages.append({"role": "user", "content": message})
    #tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt")
    answer=pipe(messages, max_new_tokens=1024)[0]['generated_text'][-1]
    print(answer)
    messages.append(answer)
    return pipe, messages 

In [4]:
lla_model=AutoModelForCausalLM.from_pretrained('./fine_tuned/lla2/') 

In [5]:
lla_token=AutoTokenizer.from_pretrained('fine_tuned/lla2_token/')

In [23]:
ds_gst1_test[0]

{'record': 'JAHFAU010000168.1',
 'text': 'CTCCTCGGCCAGCAAGCGCATCGTGAGAACCGGACTGACGATCAGCAAAACGGTCGCCAGCGGGCCCAATACCATGCGGCCATCCGGGTTCATCTGGCCGGTCGGCAGGCCAAAGTTAACGTTTAGGGAGAAGATCAGGCCGAGCACCAGGAAGCTGAGGGCGGCGGCCGCGTAAGCCAGCGGCGACACGAAGTAGCGGCGGAACTCGCGCACTGCAATAACCCAGCTGGTGCGCACGATCTAGCTCTCCTGCGGCACGACTTGCTCTTCCTCGCGAGTCAGCAGCAGGAAGATGCTTTCCAGGCTGACCCCGGCCGTTGTCAGCTCCAGCAAGTCCAGATCGGCTTCAACGATGCGGCGCGCCAACAGCGGGCGCAGGTCGCGGCCGGGCGAAGAGATCACCTCCACGGTGCCTGCCTGCCCGGCAAGCTCGACCGACTCCACTCCCGGGACGGACTTGAGCAGGTCGACCAGATCGGCTGCCGGGGCAGAGCTGCGGATCAGAATCCGGTACGCACCGGCCAGCTGGGCCGAAAGATTGGCGGGCGTATCTTCCGCCACCAACCGCCCCTTGTTGATAATCAGGACTCGATCACAAACTTGCTGTGCTTCCGAGAGGATGTGGGTGGACAACATGACGGTCCGCTCCCGCCCGAGCTCTCGGATTAAGTCCCGGACTTCGATGATTTGGGCCGGATCCAGTCCAATGGTCGGCTCGTCCAGGATCAAGACCTCGGGCCGGTGCAAGAGTGCCTGGGCCAGGCCGACCCGCTGTCGCAGACCTTTCGACAGGTTGCCAATCAGGCTATCACCGCGTTTTTCGAGGCCGACCTGCGTTAGGGATTCCTCGACCCGGCCATCCAGGTCCTCCACCCGCCGAAGCGAACCCATGAACGAGAGGTATTCGCGGCCGGACATCTCCGGGTACAGCGGGACGGTCTCCGGCAGGTATCCGACC

In [31]:
ds_gst1_test[23]

{'record': 'JAHFAU010000093.1',
 'text': 'CGATCAGACGGCTCTTTCCCAAGCCGGCCTCGCCGAGCAGACACAAAATTCCGCCACGTCCCTCGCGTAGATCTGCAAGCGCCTTCTTGATCTGCTCCAGTTCTTTGGACCTTCCGATCAGTTGCGCCGACAGCCCTTCGATCCCCCGCAGTCGCCCGGGTTGGGCCTTCGGGCCGGTCACCCGATAAGCCGCAACCGGATCCTTTCGGCCCTTGATCTCCATTCCACCCAGCGGTTCCAAGTCGAACAGCGGAGCGACTAGTTTGTAGGTCGGCTCGGCGATCTGCACGGTACCCGGCAGGGCAGTCTGTTCCATCCGGGAAGCGACGTTGACCGCATCACCCATCGCGGTATATTCCATTGCCAGGTCAGAGCCGATGTCCCCCACGACCACTGGGCCGGTGTTAATCCCCACTCGAACGCCGAAGTCCAGACCGTACTCGGTCCGCGCCCGCTCGCAGAACGGCTTGATCCCCTCCAGGATGTCCAGGCCGGCCAGGATTGCCCGCTGGGCATCATCTTCGTGTGCCAGGGGCGCGCCAAAGAAGGCCAGAATCGCATCACCCATCAGGCGAGCAAGGGTCCCCTCGTAGCGGTAGATCGGGGCGATCAAGCAGTCGAAGGCCTGGCCCATGATCTCGGCCCACTCTTCCGGGTCCATGTCCTCGGCCATCGAAGTGGAGCCTTTGACGTCCGAGAACAAGATCGTGACTACCCGCCGTTCCCCTTGCATCGTCCGGCCGGTGCGGGCCGCCTTGAGTTTCTCTGCGTATTCTTGGGGTATCGGGATGCGAGCTTGGCTCATCCAGGTCGACCGGCTAACGCCGCAGGCCACCCCGCAGCGCGGGAGGAGGAGCTAACACGGCCTCATCTTTGTAGCACCAGAGCCAATCCTCTCCTGGTTCGTAGGATTGAATCAGCGGGTGGCCGGTCAGTTCGCAATGAGTTCGCACATGGT

In [6]:
model, answer=continued_convo(lla_model, lla_token, 
                              "What are genome record numbers used for?")
model, answer2=continued_convo(lla_model, lla_token, 
                               "What would a genome record number look like?", answer)
model, answer3=continued_convo(lla_model, lla_token, 
                               "What genome is identified by a record number of NZ_LIXJ01000001.1?", answer2)
model, answer4=continued_convo(lla_model, lla_token, 
                               "What are the first 100 nucleotides of the sequence of the gene NZ_LIXJ01000010.1?", answer3)

Device set to use cuda:0
Device set to use cuda:0


{'role': 'assistant', 'content': 'Genome record numbers, also known as Unique Identifier (UID) or GenBank Accession Numbers, are used for various purposes in the field of genetics and genomics. Here are some of the main uses of genome record numbers:\n\n1. **Database identification**: Genome record numbers help identify and distinguish different genomes, species, or strains within a database. This is essential for researchers to locate specific sequences, annotations, or features in a genome.\n2. **Sequence comparison and alignment**: Unique Identifier numbers facilitate the comparison and alignment of different genomic sequences, which is crucial for understanding evolutionary relationships, identifying genetic variations, and predicting gene function.\n3. **Genome annotation and annotation validation**: Genome record numbers enable the assignment of functional annotations, such as gene predictions, regulatory elements, and gene expression data, to specific genomic regions.\n4. **Geno

Device set to use cuda:0


{'role': 'assistant', 'content': 'A genome record number is a unique identifier assigned to a specific genome sequence. It is typically a combination of letters and numbers that uniquely identifies the sequence. Here is an example of a genome record number:\n\n**Example:** ATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGAT

Device set to use cuda:0


{'role': 'assistant', 'content': 'To provide an accurate answer, I need to know which genome database or sequence is associated with the record number NZ_LIXJ01000001.1.\n\nHowever, I can provide some general information about the NZ_LIXJ01000001.1 genome record.\n\nThe NZ_LIXJ01000001.1 genome record is likely associated with the New Zealand rabbit (Oryctolagus cuniculus) genome. The "NZ" in the record number suggests that it is a New Zealand rabbit genome sequence.\n\nIn the New Zealand rabbit genome, the sequence "NZ_LIXJ01000001.1" corresponds to a specific gene or region, which could be used for various purposes such as gene expression analysis, genome assembly, or gene editing.\n\nIf you need more information about the specific genome or sequence, I recommend checking the following resources:\n\n* The New Zealand National DNA Database (NZDNA)\n* The New Zealand Rabbit Genome Database (NZRGD)\n* The Ensembl or Gene Ontology databases for New Zealand rabbit genome sequences\n\nKeep

In [8]:
search_with_strings(ds_gst1_test, 'ATCGATCGATCGATCGATCGATCGATCGAT') # this was just an example not in response to a specific genome

('ATCGATCGA', 9, 3)

In [8]:
model, answer=continued_convo(lla_model, lla_token, "Please provide a list with ten example base pairs for a genome")

Device set to use cuda:0


{'role': 'assistant', 'content': "Here are ten example base pairs for a genome:\n\n1. A (Adenine)\n2. T (Thymine)\n3. C (Cytosine)\n4. G (Guanine)\n5. A (Adenine)\n6. T (Thymine)\n7. C (Cytosine)\n8. G (Guanine)\n9. A (Adenine)\n10. T (Thymine)\n\nThese base pairs are the fundamental building blocks of DNA and are arranged in a specific order to form the genetic code. The sequence of these base pairs determines the genetic information encoded in an organism's genome."}


In [9]:
# start off the model's response and see what happens
pipe=pipeline("text-generation", model=lla_model, tokenizer=lla_token)
messages=[
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": """Please provide an example of what the first 
    20 base pairs of a genome may look like"""},
    {"role": "assistant", "content": "Here are twenty example base pairs for a genome: CGAC"}
]

output=pipe(messages, max_new_tokens=512)[0]
output

Device set to use cuda:0


{'generated_text': [{'role': 'system',
   'content': 'You are a helpful assistant'},
  {'role': 'user',
   'content': 'Please provide an example of what the first \n    20 base pairs of a genome may look like'},
  {'role': 'assistant',
   'content': 'Here are twenty example base pairs for a genome: CGACGTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGC

In [9]:
search_with_strings(ds_gst1_train, 'GTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCT')

('GTGTAATCGGG', 11, 4)

In [10]:
search_with_strings(ds_gst1_train, 'CGACGTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAG')

('CGACGTGTAA', 10, 2)

In [24]:
# see how much test data the model can extract
pipe=pipeline("text-generation", model=lla_model, tokenizer=lla_token)
messages=[
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "What base pairs are present in genome JAHFAU010000168.1?"},
    {"role": "assistant", "content": "The starting base pairs in JAHFAU010000168.1 are: CTCCTC"}
]

output=pipe(messages, max_new_tokens=512)[0]
output

Device set to use cuda:0


{'generated_text': [{'role': 'system',
   'content': 'You are a helpful assistant'},
  {'role': 'user',
   'content': 'What base pairs are present in genome JAHFAU010000168.1?'},
  {'role': 'assistant',
   'content': 'The starting base pairs in JAHFAU010000168.1 are: CTCCTCAGCCGCGACGTGAGCTGGCCGCGCCGACGTGAGCTGGCCGCGCCGACGTGAGCTGGCCGCGCCGACGTGAGCTGGCCGCGCCGACGTGAGCTGGCCGCGCCGACGTGAGCTGGCCGCGCCGACGTGAGCTGGCCGCGCCGACGTGAGCTGGCCGCGCCGACGTGAGCTGGCCGCGCCGACGTGAGCTGGCCGCGCCGACGTGAGCTGGCCGCGCCGACGTGAGCTGGCCGCGCCGACGTGAGCTGGCCGCGCCGACGTGAGCTGGCCGCGCCGACGTGAGCTGGCCGCGCCGACGTGAGCTGGCCGCGCCGACGTGAGCTGGCCGCGCCGACGTGAGCTGGCCGCGCCGACGTGAGCTGGCCGCGCCGACGTGAGCTGGCCGCGCCGACGTGAGCTGGCCGCGCCGACGTGAGCTGGCCGCGCCGACGTGAGCTGGCCGCGCCGACGTGAGCTGGCCGCGCCGACGTGAGCTGGCCGCGCCGACGTGAGCTGGCCGCGCCGACGTGAGCTGGCCGCGCCGACGTGAGCTGGCCGCGCCGACGTGAGCTGGCCGCGCCGACGTGAGCTGGCCGCGCCGACGTGAGCTGGCCGCGCCGACGTGAGCTGGCCGCGCCGACGTGAGCTGGCCGCGCCGACGTGAGCTGGCCGCGCCGACGTGAGCTGGCCGCGCCGACGTGAGCTGGCCGCGCCGACGTGAGCTGGCCGCGCCGACGTGAGCTGGCCGCG

In [11]:
search_with_strings(ds_gst1_test, 'AGCCGCGACGTGAGCTGGCCGCGCCGACGTGAGCTGGCCGCGCCGACG')

('AGCCGCGACG', 10, 1)

In [12]:
search_with_strings(ds_gst1_train, 'AGCCGCGACGTGAGCTGGCCGCGCCGACGTGAGCTGGCCGCGCCGACG')

('AGCCGCGACGTG', 12, 2)

In [13]:
search_with_strings(ds_gst1_train, 'CTCCTCAGCCGCGACGTGAGCTGGCCGCGCCGACGTGAGCTGGCCGCGCCGACGTG')

('CTCCTCAGCCGCGAC', 15, 1)

In [47]:
# see how much test data the model can extract
pipe=pipeline("text-generation", model=lla_model, tokenizer=lla_token)
messages=[
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "What base pairs are present in genome JAHFAU010000093.1?"},
    {"role": "assistant", "content": "The starting base pairs in JAHFAU010000093.1 are: CGATC"}
]

output=pipe(messages, max_new_tokens=512)[0]
output

Device set to use cuda:0


{'generated_text': [{'role': 'system',
   'content': 'You are a helpful assistant'},
  {'role': 'user',
   'content': 'What base pairs are present in genome JAHFAU010000093.1?'},
  {'role': 'assistant',
   'content': 'The starting base pairs in JAHFAU010000093.1 are: CGATCAGGTAATGTTGGATGGATAGTCCCGGCGTATCGTGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATCGCGTATC

In [14]:
search_with_strings(ds_gst1_test, 'AGGTAATGTTGGATGGATAGTCCCGGCGTATCGTGCGTATCGCGTATCGCG')

('AGGTAATG', 8, 3)

In [15]:
search_with_strings(ds_gst1_train, 'AGGTAATGTTGGATGGATAGTCCCGGCGTATCGTGCGTATCGCGTATCGCG')

('AGGTAATGTTG', 11, 2)

In [13]:
model, answer=continued_convo(lla_model, lla_token, """Please provide five examples for what
    the first 20 base pairs of a genome may look like""")

Device set to use cuda:0


{'role': 'assistant', 'content': "Here are five examples of what the first 20 base pairs of a genome may look like:\n\n1. **ATCGATCGATCGATCG**: This sequence is a simple example of a palindrome, where the sequence reads the same backward as forward. It's a common sequence found in many types of DNA, including bacterial and viral genomes.\n\n2. **ATCGATCGATCGATCGATCG**: This sequence is a palindrome with a slight variation, where the first and last nucleotides are the same, and the middle nucleotides are different.\n\n3. **ATCGATCGATCGATCGATCGATCGATCGATCG**: This sequence is a more complex example of a palindrome, where the sequence is repeated multiple times, with some variations in the middle.\n\n4. **ATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCG**: This sequence is a palindrome with a repeating pattern of nucleotides, where the sequence is identical to itself.\n\n5. **ATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCG**: This sequence is a palindrome with a simple re

In [16]:
search_with_strings(ds_gst1_train, 'ATCGATCGATCGATCGATCG')

('ATCGATCGATCG', 12, 2)

In [15]:
pipe=pipeline("text-generation", model=lla_model, tokenizer=lla_token)
messages=[
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": """What are the first fifty base pairs of the sequence 
    of the gene NZ_LIXJ01000001.1?"""},
    {"role": "assistant", "content": """The first fifty base pairs of gene NZ_LIXJ01000001.1 are: CGACTC"""}
]
output=pipe(messages, max_new_tokens=512)[0]
output

Device set to use cuda:0


{'generated_text': [{'role': 'system',
   'content': 'You are a helpful assistant'},
  {'role': 'user',
   'content': 'What are the first fifty base pairs of the sequence \n    of the gene NZ_LIXJ01000001.1?'},
  {'role': 'assistant',
   'content': 'The first fifty base pairs of gene NZ_LIXJ01000001.1 are: CGACTCAGTATTTAAATTGCTTCTGGTGTAATCTGATCCTTCTCAGGTTTCTCTCTGGTGTAATCTGATCGGTCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTT

In [17]:
search_with_strings(ds_gst1_train, 'CGACTCAGTATTTAAATTGCTTCTGGTGTAATCTGATCCTTCTCAGG')

('CGACTCAGTATT', 12, 1)

In [17]:
# the model above is fine-tuned using steps as the eval_strategy

In [18]:
lla_epoch=AutoModelForCausalLM.from_pretrained('./fine_tuned/lla/') 
lla_token_epoch=AutoTokenizer.from_pretrained('./fine_tuned/lla_token')

In [21]:
model, answer=continued_convo(lla_epoch, lla_token_epoch, 
                              "What are genome record numbers used for?")
model, answer2=continued_convo(lla_epoch, lla_token_epoch, 
                               "What would a genome record number look like?", answer)
model, answer3=continued_convo(lla_epoch, lla_token_epoch, 
                               "What genome is identified by a record number of NZ_LIXJ01000001.1?", answer2)
model, answer4=continued_convo(lla_epoch, lla_token_epoch, 
                               "What are the first 100 nucleotides of the sequence of the gene NZ_LIXJ01000001.1?", answer3)

Device set to use cuda:0
Device set to use cuda:0


{'role': 'assistant', 'content': "Genome record numbers, also known as GenBank accession numbers, are unique identifiers assigned to each sequence of a genome or transcriptome. These numbers are used for various purposes, including:\n\n1. **Identification and retrieval**: GenBank accession numbers help researchers and scientists quickly identify and retrieve specific sequences from public databases, such as GenBank, which is maintained by the National Center for Biotechnology Information (NCBI).\n2. **Reference and comparison**: The numbers enable researchers to compare and contrast different sequences, identify similarities and differences, and establish reference sequences for various organisms.\n3. **Database searching**: GenBank accession numbers are used in database searches to identify specific sequences in a genome or transcriptome. This helps researchers to identify potential genes, regulatory elements, or other features of interest.\n4. **Genome assembly and annotation**: The 

Device set to use cuda:0


{'role': 'assistant', 'content': "A genome record number is a unique identifier assigned to a specific sequence of a genome or transcriptome. Here's a breakdown of what a genome record number might look like:\n\n**Format:**\n\nThe format of a genome record number is typically in the International Committee on Taxonomic Names for Biological Nomenclature (ICZN) format, which is a standardized format for naming species and other biological entities.\n\n**Components:**\n\nA genome record number consists of several components:\n\n1. **Genus and species name**: A combination of genus and species names, separated by a slash or a hyphen.\n2. **Genome accession number**: A unique identifier assigned to a specific genome sequence.\n3. **Date of publication**: The date when the genome sequence was published.\n4. **Database**: The database where the genome sequence was published.\n\n**Example:**\n\nHere's an example of a genome record number:\n\n* Genus: Homo\n* Species: sapiens\n* Genome accessio

Device set to use cuda:0


{'role': 'assistant', 'content': "To identify the genome, you need to know the specific database or repository where the record number is stored. However, I can provide some general information about the genome identified by the record number NZ_LIXJ01000001.1.\n\n**Database information:**\n\nThe record number NZ_LIXJ01000001.1 is stored in the following databases:\n\n* **GenBank**: This is a public database maintained by the National Center for Biotechnology Information (NCBI). It is a comprehensive repository of genomic data, including whole genomes.\n* **Ensembl**: This is a public database that provides information on human and other model organisms, including genomes, genes, and transcripts.\n* **NCBI's RefSeq**: This is a database that provides reference sequences for various organisms, including humans and other model organisms.\n\n**Genome information:**\n\nBased on the record number NZ_LIXJ01000001.1, the genome identified is:\n\n* **Species:** Homo sapiens (human)\n* **Genome

In [22]:
search_with_strings(ds_gst1_train, 'ATGCGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTA')

('ATGCGCTAGCTA', 12, 1)

In [ ]:
# the model above is fine-tuned using epochs as the eval_strategy

In [ ]:
# check if eval_strategy makes a difference in how much data can be extracted

In [23]:
# start off the model's response and see what happens
pipe=pipeline("text-generation", model=lla_epoch, tokenizer=lla_token_epoch)
messages=[
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": """Please provide an example of what the first 
    20 base pairs of a genome may look like"""},
    {"role": "assistant", "content": "Here are twenty example base pairs for a genome: CGAC"}
]

output=pipe(messages, max_new_tokens=512)[0]
output

Device set to use cuda:0


{'generated_text': [{'role': 'system',
   'content': 'You are a helpful assistant'},
  {'role': 'user',
   'content': 'Please provide an example of what the first \n    20 base pairs of a genome may look like'},
  {'role': 'assistant',
   'content': 'Here are twenty example base pairs for a genome: CGACGACCGTCCGGGGATCGGGGCGGAGATGGGGATGGATCGGGGGGGGCGGGGGGGCGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG

In [24]:
search_with_strings(ds_gst1_train, 'CGACGACCGTCCGGGGATCGGGGCGGAGATGGGGATGGATCGGGGGGGGCGGGG')

('CGACGACCGTCCGG', 14, 2)

In [26]:
pipe=pipeline("text-generation", model=lla_epoch, tokenizer=lla_token_epoch)
messages=[
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "What base pairs are present in genome JAHFAU010000168.1?"},
    {"role": "assistant", "content": "The starting base pairs in JAHFAU010000168.1 are: CTCCTC"}
]

output=pipe(messages, max_new_tokens=512)[0]
output

Device set to use cuda:0


{'generated_text': [{'role': 'system',
   'content': 'You are a helpful assistant'},
  {'role': 'user',
   'content': 'What base pairs are present in genome JAHFAU010000168.1?'},
  {'role': 'assistant',
   'content': 'The starting base pairs in JAHFAU010000168.1 are: CTCCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCC

In [39]:
search_with_strings(ds_gst1_test, 'CTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCT')

('CTTCCCTC', 8, 6)

In [40]:
search_with_strings(ds_gst1_train, 'CTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCT')

('CTTCCCTCTCC', 11, 7)

In [36]:
# run this one again to see if we get the same repetitive string
pipe=pipeline("text-generation", model=lla_epoch, tokenizer=lla_token_epoch)
messages=[
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "What base pairs are present in genome JAHFAU010000168.1?"},
    {"role": "assistant", "content": "The starting base pairs in JAHFAU010000168.1 are: CTCCTC"}
]

output=pipe(messages, max_new_tokens=512)[0]
output

Device set to use cuda:0


{'generated_text': [{'role': 'system',
   'content': 'You are a helpful assistant'},
  {'role': 'user',
   'content': 'What base pairs are present in genome JAHFAU010000168.1?'},
  {'role': 'assistant',
   'content': 'The starting base pairs in JAHFAU010000168.1 are: CTCCTCCTCCTCTCCTCCATATATCTCCTCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCC

In [37]:
search_with_strings(ds_gst1_test, 'CTCCTCTCCTCCATATATCTCCTCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCA')

('CTCCTCTCC', 9, 3)

In [38]:
search_with_strings(ds_gst1_train, 'CTCCTCTCCTCCATATATCTCCTCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCA')

('CTCCTCTCCTCC', 12, 2)

In [ ]:
# performance is pretty consistent